## 1. 크롤링과 데이터베이스 예제

* cine21 배우 랭킹 사이트 크롤링
  - 사이트 주소: http://www.cine21.com/rank/person/
  - 요청 방식 확인 방법: 크롬 개발자 모드로 들어가서, Network -> content 페이지의 요청 방식 확인
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form data
      - section = 'actor'
      - period_start = '2018-08'
      - gender = 'all'
      - page = 3 

#### 1. 라이브러리 import

In [7]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

#### 2. mongodb connection

In [2]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

#### 3. 크롤링 주소 requests

In [4]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

#### 4. parsing 과 배우 이름 추출

In [5]:
soup = BeautifulSoup(res.content, 'html.parser')

In [6]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print (actor.text)

주지훈(6편)
진선규(10편)
마동석(10편)
류승룡(5편)
하정우(8편)
이하늬(4편)
송강호(12편)


#### 정규 표현식 참고: https://www.fun-coding.org/DS&AL3-4.html

In [8]:
test_data = '주지훈(6편)'
re.sub('\(\w*\)', '', test_data)

'주지훈'

In [10]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print (re.sub('\(\w*\)', '', actor.text))

주지훈
진선규
마동석
류승룡
하정우
이하늬
송강호


#### 5. 배우 상세 정보 추출

#### 참고: 특수한 정규 표현식
```html
  - Greedy(.*) vs Non-Greedy(.*?)
  - 연습: https://regexr.com
    - <li><span class="tit">원어명</span>주지훈</li>
  - . 문자는 줄바꿈 문자인 \n 를 제외한 모든 문자 한개를 의미함
  - * 는 앞 문자가 0번 또는 그 이상 반복되는 패턴
  - .* 는 문자가 0번 또는 그 이상 반복되는 패턴
```

In [27]:
actors = soup.select('li.people_li div.name')

actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    actor_info_dict = dict()
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value
    actors_info_list.append(actor_info_dict)

actors_info_list

[{'원어명': '주지훈',
  '다른 이름': '주영훈',
  '직업': '배우',
  '생년월일': '1982-05-16',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n',
  '신장/체중': '187cm, 68kg',
  '학교': '경기대학교 연기',
  '취미': '컴퓨터 게임',
  '특기': '속독, 노래, 춤, 합기도, 피아노'},
 {'직업': '배우', '생년월일': '1977-09-13', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1971-03-01',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n'},
 {'다른 이름': '류승용; 유승룡',
  '직업': '배우',
  '생년월일': '1970-11-29',
  '성별': '남',
  '신장/체중': '175cm, 72kg',
  '학교': '서울예술대학 연극',
  '소속사': '필름있수다'},
 {'다른 이름': '김성훈; 河正宇',
  '직업': '배우',
  '생년월일': '1978-03-11',
  '성별': '남',
  '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n',
  '신장/체중': '184cm, 75kg',
  '학교': '중앙대학교 연극학 학사',
  '취미': '피아노, 검도, 수영'},
 {'직업': '배우',
  '생년월일': '1983-03-02',
  '성별': '여',
  '홈페이지': '\nhttps://www.facebook.com/saram.leehoney\nhttps://instagram.com/honey_lee32/\n'},
 {'직업': '배우',
  '생년월일': '1967-01-17',
  '성별': '남',
  '신장/체중': '180cm, 80kg',
  '학교': '부

#### 6. 배우 흥행 지수와 출연 영화 추출

In [37]:
actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
for index, actor in enumerate(actors):
    print ("배우이름:", re.sub('\(\w*\)', '', actor.text))
    print ("흥행지수:", int(hits[index].text.replace(',', '')))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    print ("출연영화:", movie_title_list)



배우이름: 주지훈
흥행지수: 29515
출연영화: ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들']
배우이름: 진선규
흥행지수: 27687
출연영화: ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '완벽한 타인']
배우이름: 마동석
흥행지수: 25252
출연영화: ['신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물', '악인전']
배우이름: 류승룡
흥행지수: 21331
출연영화: ['극한직업', '명량', '표적', '광해, 왕이 된 남자', '열혈남아']
배우이름: 하정우
흥행지수: 21128
출연영화: ['신과 함께-인과 연', 'PMC: 더 벙커', '신과 함께-죄와 벌', '더 테러 라이브', '암살', '베를린']
배우이름: 이하늬
흥행지수: 19597
출연영화: ['극한직업', '연가시', '달빛궁궐', '조작된 도시']
배우이름: 송강호
흥행지수: 18440
출연영화: ['마약왕', '기생충', '나랏말싸미', '박쥐', '복수는 나의 것', '설국열차']


#### 7. 수집한 데이터 기반 사전 만들기

In [47]:
actors_info_list = list()

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
rankings = soup.select('li.people_li > span.grade')

for index, actor in enumerate(actors):
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_hits = int(hits[index].text.replace(',', ''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    actor_info_dict = dict()
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_title_list
    actor_info_dict['랭킹'] = rankings[index].text
    
    actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value
    actors_info_list.append(actor_info_dict)
    print (actor_info_dict)

actors_info_list

{'배우이름': '주지훈', '흥행지수': 29515, '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들'], '랭킹': '1', '원어명': '주지훈', '다른 이름': '주영훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노'}
{'배우이름': '진선규', '흥행지수': 27687, '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '완벽한 타인'], '랭킹': '2', '직업': '배우', '생년월일': '1977-09-13', '성별': '남'}
{'배우이름': '마동석', '흥행지수': 25252, '출연영화': ['신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물', '악인전'], '랭킹': '3', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n'}
{'배우이름': '류승룡', '흥행지수': 21331, '출연영화': ['극한직업', '명량', '표적', '광해, 왕이 된 남자', '열혈남아'], '랭킹': '4', '다른 이름': '류승용; 유승룡', '직업': '배우', '생년월일': '1970-11-29', '성별': '남', '신장/체중': '175cm, 72kg', '학교': '서울예술대학 연극', '소속사': '필름있수다'}
{'배우이름': '하정우', '흥행지수': 21128, '출연영화': ['신과 함께-인과 연

[{'배우이름': '주지훈',
  '흥행지수': 29515,
  '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들'],
  '랭킹': '1',
  '원어명': '주지훈',
  '다른 이름': '주영훈',
  '직업': '배우',
  '생년월일': '1982-05-16',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n',
  '신장/체중': '187cm, 68kg',
  '학교': '경기대학교 연기',
  '취미': '컴퓨터 게임',
  '특기': '속독, 노래, 춤, 합기도, 피아노'},
 {'배우이름': '진선규',
  '흥행지수': 27687,
  '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '완벽한 타인'],
  '랭킹': '2',
  '직업': '배우',
  '생년월일': '1977-09-13',
  '성별': '남'},
 {'배우이름': '마동석',
  '흥행지수': 25252,
  '출연영화': ['신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물', '악인전'],
  '랭킹': '3',
  '직업': '배우',
  '생년월일': '1971-03-01',
  '성별': '남',
  '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n'},
 {'배우이름': '류승룡',
  '흥행지수': 21331,
  '출연영화': ['극한직업', '명량', '표적', '광해, 왕이 된 남자', '열혈남아'],
  '랭킹': '4',
  '다른 이름': '류승용; 유승룡',
  '직업': '배우',
  '생년월일': '1970-11-29',
  '성별': '남',
  '신장/체중': '175cm, 72kg',
  '학교': '서울

#### 8. 여러 페이지의 배우 상세 정보 추출하기

In [48]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'

for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)
        
print (actors_info_list)

[{'배우이름': '주지훈', '흥행지수': 29515, '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들'], '랭킹': '1', '원어명': '주지훈', '다른 이름': '주영훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노'}, {'배우이름': '진선규', '흥행지수': 27687, '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '완벽한 타인'], '랭킹': '2', '직업': '배우', '생년월일': '1977-09-13', '성별': '남'}, {'배우이름': '마동석', '흥행지수': 25252, '출연영화': ['신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물', '악인전'], '랭킹': '3', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n'}, {'배우이름': '류승룡', '흥행지수': 21331, '출연영화': ['극한직업', '명량', '표적', '광해, 왕이 된 남자', '열혈남아'], '랭킹': '4', '다른 이름': '류승용; 유승룡', '직업': '배우', '생년월일': '1970-11-29', '성별': '남', '신장/체중': '175cm, 72kg', '학교': '서울예술대학 연극', '소속사': '필름있수다'}, {'배우이름': '하정우', '흥행지수': 21128, '출연영화': ['신과 함께

In [43]:
for index in range(20):
    print (index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


#### 9. mongodb에 크롤링 데이터 저장하기

In [49]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.insert_many(actors_info_list)

In [50]:
actor_collection.delete_many({})

#### 10. 배우 랭킹 및 상세 정보 크롤링 및 mongodb 저장

In [51]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'

for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)
        
actor_collection.insert_many(actors_info_list)